In [54]:
import pandas as pd
from pandas import ExcelWriter 
pd.__version__

'1.0.3'

In [80]:
class JTLReader:
    def __init__(self,jtlPath):
        self.jtlPath=jtlPath
        self.jtlColumns = ['timeStamp', 'elapsed','label','bytes','allThreads','responseCode']
        %time self.df_Jtl=pd.read_csv(self.jtlPath,usecols=self.jtlColumns,dtype={'label':'category'})#,nrows=100     
        self.shape=self.df_Jtl.shape
        self.rows=self.shape[0]
        self.rows 
        #self.df_Jtl.truncate(before="A", after="B", axis="columns")
       
        
    def converTimeStamp(self):
        self.df_Jtl['timeStamp'] = pd.to_datetime(self.df_Jtl['timeStamp'],unit='ms')
        self.df_Jtl['timeStamp'] = self.df_Jtl['timeStamp'].astype('datetime64[s]')#reduce precision pandas timestamp dataframe
        self.df_Jtl['timeStamp']=self.df_Jtl['timeStamp'].dt.tz_localize('utc').dt.tz_convert('Singapore')
        self.df_Jtl['timeStamp'] = self.df_Jtl['timeStamp'].astype(str).str[:-6]
        self.df_Jtl['timeStamp'] = pd.to_datetime(self.df_Jtl['timeStamp'])
        self.df_Jtl['timeStamp'] = self.df_Jtl['timeStamp'].astype('datetime64[s]') 
        
    def ms_to_sec(self):
         self.df_Jtl['elapsed']=(self.df_Jtl['elapsed']/1000)  
        
    def overAllResponseTimeSummary(self):
        self.df_overAllResponseTimeSummary=self.df_Jtl[['timeStamp','elapsed']]
        return self.df_overAllResponseTimeSummary
    
    def responseTimeSummary(self):
        self.df_responseTimeSummary=self.df_Jtl[['label','elapsed']]
        self.df_txnGroup=self.df_responseTimeSummary.groupby(['label'])
        self.df_txnGroup=self.df_txnGroup.elapsed.describe(percentiles=[.90])#self.df_txnGroup.describe(percentiles=[.90])
        self.df_txnGroup.columns=self.df_txnGroup.columns.get_level_values(0)#self.df_txnGroup.columns=self.df_txnGroup.columns.get_level_values(1)
        self.df_txnGroup = self.df_txnGroup[['min','mean','90%','max','count']]
        #self.df_txnGroup=self.df_txnErrorGroup.reset_index() """To be enable if you run describe () on data frame. Now we are running on specific column"""
        return self.df_txnGroup
    
    def errorSummary(self):
        self.df_errorSummary=self.df_Jtl[['label','responseCode']]
        self.df_txnErrorGroup=self.df_errorSummary.groupby(['label'])
        self.df_txnErrorGroup=self.df_txnErrorGroup.responseCode.describe()
        return self.df_txnErrorGroup
    
        
    def calculateErrorPercentage(self,responseTimeSummary,errorSummary):
        #Method 1
        self.txn_grp=self.df_Jtl.groupby(['label'])
        #return self.txn_grp.get_group('BP01_TX05_Submit')
        
        #Method 2
        #filt=self.df_Jtl['label']=='BP01_TX05_Submit'
        #return self.df_Jtl.loc[filt]['responseCode'].value_counts()
        
        #Method 3
        #self.txn_grp=self.df_Jtl.groupby(['label'])
        #return self.txn_grp['responseCode'].value_counts(normalize=True)
        
        #self.df_responseTimeSummary=responseTimeSummary
        #self.df_errorSummary=errorSummary
        #self.response_error=pd.merge(self.df_responseTimeSummary,self.df_errorSummary)
        # return self.response_error
        return self.txn_grp.get_group('DS53_Bids - Bid Validation Status Retrieval')
        
        
        
        return self.txn_grp
    
    def listOfErrors(self):
        return self.df_Jtl['responseCode'].value_counts()
    
    def shapeOfDaraFrame(self):
        return self.df_Jtl.shape
    
    def columnType(self,columnName):
        self.columnName=columnName
        return self.df_Jtl[self.columnName].dtype
    


       
        
    
        
obj1=JTLReader("C:\\Karthik\\Pandas\\14novtest.jtl")
obj1.converTimeStamp()
obj1.ms_to_sec() 



Wall time: 3.41 s


In [30]:
obj1.shapeOfDaraFrame()

(3535050, 6)

In [72]:
obj1.listOfErrors()

200                                                        3010352
200                                                         524119
500                                                            410
Non HTTP response code: java.net.SocketException                96
500                                                             63
Non HTTP response code: java.net.ConnectException                6
Non HTTP response code: java.net.SocketTimeoutException          4
Name: responseCode, dtype: int64

In [4]:
#overAllResponseTimeSummary
obj1.overAllResponseTimeSummary()


,timeStamp,elapsed
0,2018-11-14 11:46:36,1.073
1,2018-11-14 11:46:36,1.271
2,2018-11-14 11:46:36,1.954
3,2018-11-14 11:46:36,2.036
4,2018-11-14 11:46:36,2.184
...,...,...
3535045,2018-11-14 20:01:34,0.853
3535046,2018-11-14 20:01:34,0.176
3535047,2018-11-14 20:01:34,0.919
3535048,2018-11-14 20:01:34,0.441


In [5]:
#responseTimeSummary
responseTimeSummary=obj1.responseTimeSummary()
responseTimeSummary


,min,mean,90%,max,count
label,,,,,
DS01_AdvisoriesRetrieval,0.130,33.692795,48.8366,74.528,1762.0
DS02_PenaltyReports - PEN001,0.195,0.904070,1.4600,301.733,65656.0
DS04_PenaltyReports - PEN012,0.455,2.968362,4.4859,304.929,20002.0
DS05_PenaltyReports - PEN010,0.208,0.883374,1.4130,303.553,67199.0
DS06_CompensationReports - COM001,0.195,0.874749,1.3980,302.059,67858.0
DS07_CompensationReports - COM002,0.217,1.096928,1.7200,301.139,54121.0
DS08_CompensationReports - COM010,0.171,0.842357,1.3470,602.942,70464.0
DS09_Invoice Reports - STL_INV,8.701,39.834209,69.1272,356.829,1489.0
DS10_SettlementReports - STL001,0.100,0.508164,0.9320,13.689,116778.0


In [7]:
#write response summary to excel with txn name,min,avg,max,90% & max
responseTimeSummary=obj1.responseTimeSummary()
writer = ExcelWriter('PythonExport.xlsx')
responseTimeSummary.to_excel(writer,sheet_name='ResponseSummary')
writer.save()

In [19]:
errorSummary=obj1.errorSummary()
errorSummary

,count,unique,top,freq
label,,,,
DS01_AdvisoriesRetrieval,1762,3,200,1450
DS02_PenaltyReports - PEN001,65656,4,200,56735
DS04_PenaltyReports - PEN012,20002,3,200,17101
DS05_PenaltyReports - PEN010,67199,3,200,57992
DS06_CompensationReports - COM001,67858,3,200,58810
DS07_CompensationReports - COM002,54121,3,200,46966
DS08_CompensationReports - COM010,70464,4,200,60966
DS09_Invoice Reports - STL_INV,1489,2,200,1223
DS10_SettlementReports - STL001,116778,3,200,98796


In [44]:
ser=obj1.calculateErrorPercentage(responseTimeSummary,errorSummary)
ser

label                         responseCode
DS01_AdvisoriesRetrieval      200             0.822928
                              200             0.175936
                              500             0.001135
DS02_PenaltyReports - PEN001  200             0.864125
                              200             0.135814
                                                ...   
DS57_Bids - Pre Validate      200             0.134318
DS58_Bids - Get Batch Status  200             0.888365
                              200             0.111635
Ds03_PenaltyReports - PEN002  200             0.865226
                              200             0.134774
Name: responseCode, Length: 151, dtype: float64

In [50]:
ser1=obj1.calculateErrorPercentage(responseTimeSummary,errorSummary)
writer = ExcelWriter('PythonExport.xlsx')
ser1.to_excel(writer,sheet_name='DS53_BidsBidValidatio')
writer.save()

In [45]:
df=ser.unstack()
df

responseCode,200,500,200,500,Non HTTP response code: java.net.ConnectException,Non HTTP response code: java.net.SocketException,Non HTTP response code: java.net.SocketTimeoutException
label,,,,,,,
DS01_AdvisoriesRetrieval,0.822928,0.001135,0.175936,NaN,NaN,NaN,NaN
DS02_PenaltyReports - PEN001,0.864125,NaN,0.135814,NaN,0.000015,0.000046,NaN
DS04_PenaltyReports - PEN012,0.854965,NaN,0.144936,NaN,NaN,0.000100,NaN
DS05_PenaltyReports - PEN010,0.862989,NaN,0.136951,NaN,NaN,0.000060,NaN
DS06_CompensationReports - COM001,0.866663,NaN,0.133278,NaN,NaN,0.000059,NaN
DS07_CompensationReports - COM002,0.867796,NaN,0.132130,NaN,NaN,0.000074,NaN
DS08_CompensationReports - COM010,0.865208,0.000014,0.134679,NaN,NaN,0.000099,NaN
DS09_Invoice Reports - STL_INV,0.821357,NaN,0.178643,NaN,NaN,NaN,NaN
DS10_SettlementReports - STL001,0.846016,NaN,0.153925,NaN,NaN,0.000060,NaN


In [307]:
df.loc[:,'Non HTTP response code: org.apache.http.conn.HttpHostConnectException']

label
/-1                                NaN
/login.php-4                       NaN
/mercurypurchase.php-11            NaN
/mercurypurchase2.php-16      0.005929
/mercuryreservation2.php-8    0.001972
/mercurysignoff.php-23        0.003968
BP01_TX01_Launch URL               NaN
BP01_TX02_Login                    NaN
BP01_TX03_Continue1           0.001972
BP01_TX04_Continue2                NaN
BP01_TX05_Logoff              0.003968
BP01_TX05_Submit              0.005929
DeleteUserName                     NaN
GetUserName                        NaN
Name: Non HTTP response code: org.apache.http.conn.HttpHostConnectException, dtype: float64

In [14]:
obj1.calculateErrorPercentage()

TypeError: calculateErrorPercentage() missing 2 required positional arguments: 'responseTimeSummary' and 'errorSummary'

In [81]:
temp=obj1.columnType('responseCode')
temp


dtype('O')